In [81]:
import random
import pandas as pd
import re
import pickle
import ir_datasets
import time
from rerankers import Reranker
from rank_bm25 import BM25Okapi

# **Split em conjuntos**

Datasets de treino e teste

In [ ]:
random.seed(42)

def load_dataset(input_file):
    with open(input_file, 'rb') as f:
        return pickle.load(f)

data_set = "../subset_msmarco_train_0/subset_msmarco_train_0.01_99.pkl"

data = load_dataset(data_set)
queries = data["queries"]

# Split the queries (assuming queries is a dictionary of {query_id: query_object})
query_ids = list(queries.keys())  # List of query IDs

# Shuffle query IDs to ensure a random split
random.shuffle(query_ids)

# Split into 80% for training, 20% for validation
split_ratio = 0.8
train_query_ids = query_ids[:int(len(query_ids) * split_ratio)]
test_query_ids = query_ids[int(len(query_ids) * split_ratio):]

train_queries = {qid: queries[qid] for qid in train_query_ids}
test_queries = {qid: queries[qid] for qid in test_query_ids}

# **Tratamento**

In [83]:
# Dicionário agrupa os documentos relevantes por consulta (query)
relevant_docs = dict()

for qrel in data["qrels"]:
    relevant_docs[qrel.query_id] = relevant_docs.get(qrel.query_id, []) + [qrel.doc_id]

# e.g.: relevant_docs = {'query_id': ['doc_id_1', 'doc_id_12']}

In [84]:
# Identifica os documentos relevantes no conjunto de treinamentom 
# removendo duplicatas
train_docs = set()
for qid in train_query_ids:
    train_docs.update(relevant_docs[qid])

The **mean reciprocal rank** is a statistic measure for evaluating any process that produces a list of possible responses to a sample of queries, ordered by probability of correctness. The reciprocal rank of a query response is the multiplicative inverse of the rank of the first correct answer: $1$ for first place, $\frac{1}{2}$ for second place, $\frac{1}{3}$ for third place and so on. The mean reciprocal rank is the average of the reciprocal ranks of results for a sample of queries Q:

$$
MRR =
\frac{1}{\|Q\|} = \sum_{i=1}^{\|Q\|} \frac{1}{rank_i}
$$

where ${\displaystyle {\text{rank}}_{i}}$ refers to the rank position of the first relevant document for the i-th query.

The reciprocal value of the mean reciprocal rank corresponds to the harmonic mean of the ranks.

In [85]:
def calculateMRR(query_id, ranking):
    """
    Calcula o MRR (Mean Reciprocal Rank) para uma consulta.
    
    :param query_id: ID da query atual.
    :param ranking: Lista de documentos recuperados ordenados.
    
    :return: MRR para a consulta.
    """

    for i, doc in enumerate(ranking):
        if doc[0] in relevant_docs[query_id]:
            return 1 / (i + 1)
    
    return 0

In [86]:
def getRelevantDocTexts(query_id):
    """
    Retorna os textos dos documentos relevantes para uma consulta.
    
    :param query_id: ID da query.
    
    :return: Lista de textos dos documentos relevantes.
    """
    
    relevant_doc_texts = []
    for doc_id in relevant_docs[query_id]:
        relevant_doc_texts.append(data["docs"][doc_id].text)

    return relevant_doc_texts

In [87]:
def preprocess(text, remove_punctuation = True, lowercase = True):
    """
    Função para preprocessamento de texto: remove pontuação, converte para minúsculas e divide em tokens.
    
    :param text: Texto a ser processado.
    :remove_punctuation bool: Para True remove pontuação
    :lowercase bool: Para True coloca em lowercase

    :return: Lista de tokens do texto.
    """

    # [^\w\s] corresponde a qualquer caractere que não seja uma letra, número, underscore ou espaço em branco
    # mantendo apenas letras, números e espaços

    # Caso 1
    if remove_punctuation:
        text = re.sub(r"[^\w\s]", "", text)
    
    # Caso 2
    if lowercase:
        text = text.lower()

    return text.split()

# **Modelo baseline**

## **BM25**

O BM25Okapi é uma implementação do BM25, um modelo clássico utilizado em sistemas de recuperação de informações (como motores de busca) para avaliar a relevância de documentos em relação a uma consulta (query).

O BM25 é uma fórmula de pontuação de relevância que calcula o quão bem um documento corresponde a uma consulta com base nas palavras que o documento contém.

Ele é um modelo probabilístico de recuperação de informações, baseado na ideia de que a relevância de um documento para uma consulta depende da frequência das palavras que aparecem no documento e na consulta, mas com uma diminuição das contribuições das palavras que ocorrem com frequência excessiva.

In [88]:
tokenized_corpus = [preprocess(doc.text) for doc in data["docs"].values()]
bm25 = BM25Okapi(tokenized_corpus)

In [89]:
def baseline_model(query_id, remove_punctuation, lowercase):
    query = queries[query_id].text
    tokenized_query = preprocess(query, remove_punctuation, lowercase)

    doc_scores = bm25.get_scores(tokenized_query)
    
    doc_ranking = sorted(zip(data["docs"].keys(), doc_scores), key=lambda x: x[1], reverse=True)

    return doc_ranking

In [90]:
bm_25_results = pd.DataFrame(columns=("Dataset", "Size", "MRR", "Runtime (sec)"))

Dataset de teste

In [91]:
average_mrr = 0

start_time = time.time()
for query_id in test_query_ids:
    doc_ranking = baseline_model(query_id, True, True)
    average_mrr += calculateMRR(query_id, doc_ranking)

average_mrr /= len(test_query_ids)
end_time = time.time()
execution_time = end_time - start_time

bm_25_results.loc[0] = ["Test", len(test_query_ids), average_mrr, execution_time]

Dataset de treino

In [92]:
average_mrr = 0

start_time = time.time()
for query_id in train_query_ids:
    doc_ranking = baseline_model(query_id, True, True)
    average_mrr += calculateMRR(query_id, doc_ranking)

average_mrr /= len(train_query_ids)
end_time = time.time()
execution_time = end_time - start_time

bm_25_results.loc[1] = ["Train", len(train_query_ids), average_mrr, execution_time]

Dataset completo

In [93]:
average_mrr = 0

start_time = time.time()
for query_id in query_ids:
    doc_ranking = baseline_model(query_id, True, True)
    average_mrr += calculateMRR(query_id, doc_ranking)

average_mrr /= len(query_ids)
end_time = time.time()
execution_time = end_time - start_time

bm_25_results.loc[2] = ["Full", len(query_ids), average_mrr, execution_time]

Resultado recente:
- Com processamento dos documentos
- Com processamento das queries

Obs.: O processamento inclui remover qualquer caractere que não seja uma letra, número, underscore ou espaço em branco; e deixar em lowercase.

In [94]:
print("==> Estudo de escalabilidade:")
bm_25_results

==> Estudo de escalabilidade:


,Dataset,Size,MRR,Runtime (sec)
0,Test,555,0.488720,298.350545
1,Train,2216,0.449240,1220.195126
2,Full,2771,0.457147,1467.348227


In [ ]:
bm_25_results.to_csv("../results/bm_25_results.txt")

Resultado anterior:
- Com processamento dos documentos
- Sem processamento das queries (apenas lowercase)

In [ ]:
print("==> Estudo de escalabilidade:")
bm_25_results

,Dataset,Size,MRR,Runtime
0,Test,555,0.474292,257.141707
1,Train,2216,0.428867,1072.796115
2,Full,2771,0.437965,1340.438707


Ablação na etapa de tratamento dos dados:

In [98]:
bm_25_processing_results = pd.DataFrame(columns=("Remove punctuation", "Lowercase", "MRR", "Runtime (sec)"))
index = 0

for remove_punctuation in [True, False]:
    for lowercase in [True, False]:
        average_mrr = 0
        
        start_time = time.time()
        for query_id in test_query_ids:
            doc_ranking = baseline_model(query_id, remove_punctuation, lowercase)
            average_mrr += calculateMRR(query_id, doc_ranking)

        average_mrr /= len(test_query_ids)
        end_time = time.time()
        execution_time = end_time - start_time


        bm_25_processing_results.loc[index] = [remove_punctuation, lowercase, average_mrr, execution_time]

        index += 1

In [99]:
print("==> Estudo do desempenho do tratamento de dados:")
bm_25_processing_results

==> Estudo do desempenho do tratamento de dados:


,Remove punctuation,Lowercase,MRR,Runtime (sec)
0,True,True,0.488720,297.235475
1,True,False,0.488860,289.510962
2,False,True,0.474292,297.298308
3,False,False,0.474432,288.388891


In [ ]:
bm_25_processing_results.to_csv("../results/bm_25_processing_results.txt")

# **Modelo Reranker**

In [77]:
ranker = Reranker("cross-encoder", device='cuda')
def model(query_id, remove_punctuation, lowercase):
    query = queries[query_id].text
    tokenized_query = preprocess(query, remove_punctuation, lowercase)
    doc_scores = bm25.get_scores(tokenized_query)
    doc_ranking = sorted(zip(data["docs"].keys(), doc_scores), key=lambda x: x[1], reverse=True)
    top_10 = doc_ranking[:10]
    top_10_ids = [doc_id for doc_id, score in top_10]
    top_10_texts = [data["docs"][doc_id].text for doc_id in top_10_ids]
    reranked = ranker.rank(query=query, docs=top_10_texts, doc_ids=top_10_ids)
    doc_ids = [result.doc_id for result in reranked]
    scores = [result.score for result in reranked]
    doc_ranking = list(zip(doc_ids, scores))

    return doc_ranking

Loading default cross-encoder model for language en
If your model is NOT intended to be ran as a one-label cross-encoder, please reload it and specify the model_type! Otherwise, you may ignore this warning. You may specify `model_type='cross-encoder'` to suppress this warning in the future.
Default Model: mixedbread-ai/mxbai-rerank-base-v1
Loading TransformerRanker model mixedbread-ai/mxbai-rerank-base-v1 (this message can be suppressed by setting verbose=0)
No dtype set
Using dtype torch.float32
Loaded model mixedbread-ai/mxbai-rerank-base-v1
Using device cuda.
Using dtype torch.float32.


### Inferência

In [78]:
reranker_results = pd.DataFrame(columns=("Dataset", "Size", "MRR", "Runtime"))

In [80]:
average_mrr2 = 0

start_time = time.time()
for query_id in test_query_ids:
    doc_ranking = model(query_id, True, True)
    average_mrr2 += calculateMRR(query_id, doc_ranking)
    
average_mrr2 /= len(test_query_ids)
end_time = time.time()

execution_time = end_time - start_time

reranker_results.loc[0] = ["Test", len(test_query_ids), average_mrr2, execution_time]

Resultado recente:
- Com processamento dos documentos
- Com processamento das queries

Obs.: O processamento inclui remover qualquer caractere que não seja uma letra, número, underscore ou espaço em branco; e deixar em lowercase.

In [97]:
reranker_results

,Dataset,Size,MRR,Runtime
0,Test,555,0.593363,312.731814


Resultado anterior:
- Com processamento dos documentos
- Sem processamento das queries (apenas lowercase)

In [75]:
reranker_results

,Dataset,Size,MRR,Runtime
0,Test,555,0.582853,315.65561


In [ ]:
reranker_results.to_csv("../results/reranker_results.txt")